In [228]:
import pandas as pd
import numpy as np

In [229]:
print("Loading Resident's Data.....\n")
df_residents = pd.read_excel('../Data/1609154 Resident updated List of The World Spa Jan -2020 (2).xlsx')
df_visitors = pd.read_csv('../Data/Wspa Visitor 1406-2106.csv')
df_service = pd.read_csv('../Data/Wspa service staff.csv')
df_complaints = pd.read_csv('../Data/Complaints.csv')
print("Done")

Loading Resident's Data.....

Done


In [230]:
#Storing Occupied Households info in a list

#list comprises of 'Tower', 'Apt Num', Names...., Ages...., external entitites tba later

#Randomising covid data for houses with probabilistic randomisation
res_positive = np.random.choice([0, 1], p=[0.98, 0.02], size=len(df_residents))

res_info = []

for i in range(len(df_residents)):
    if ((df_residents.iloc[i,2]=='Y' or df_residents.iloc[i,2]=='T') and (df_residents.iloc[i,0]!='SW' or df_residents.iloc[i,0]!='SE')):  
        list_house = []
        list_house.append(df_residents.iloc[i,0])
        list_house.append(df_residents.iloc[i,1])
        # Append Names of residents
        for j in range(int(df_residents.iloc[i,5])):  #Total members in the house
            list_house.append(df_residents.iloc[i,6+(4*j)])
        #Append ages of all residents   
        for j in range(int(df_residents.iloc[i,5])):  
            list_house.append(df_residents.iloc[i,7+(4*j)])
        
#         list_house.append(df_residents.iloc[i,-1])
            
        res_info.append(list_house)

In [231]:
#Just Corrected my name in the dataset!
res_info[266][4] = 'RITVIK VIJ'

In [242]:
print(res_info[0])

['A1', 101.0, 'PAVAN CHOUDARY', 'ANU CHOUDARY', 'SIDHARTH CHOUDARY', 'GAUTAM CHOUDARY', 'BINDA CHOUDARY', 53.0, 52.0, 27.0, 24.0, 83.0]


In [232]:
#Construct fully connected graph for all households
#Node number is index of household in res_info

#Point system depicting points issued by index household to the other houses
#100 points for +ve case in the house, 50 points for immediate neighbours, 10 points per household in tower, 10 points for elderly in house

inter_points = np.zeros((len(res_info), len(res_info)))

for i in range(len(res_info)):
    if (res_positive[i]):
        #identify same towers
        indices = [j for j in range(len(res_info)) if res_info[j][0]==res_info[i][0]]    
        inter_points[i][indices] = 10
        
        #Assign 100 points to the affected house
        inter_points[i][i] = 100
        
        #Assign 50 points to immediate neighbour
        neg = [j for j in indices if np.absolute(res_info[j][1]-res_info[i][1])==1.]
        inter_points[i][neg] = 50
        
    else:
        ages = np.asarray(res_info[i][int((len(res_info[i])/2)+1):-1])
        ages = ages[~np.isnan(ages)]
        
        if (len(np.where(ages>60.))):
            inter_points[i][i] = 10

In [233]:
#Accumulated points by each household
res_points = np.sum(inter_points, axis=0)

In [234]:
#Limit points to 100
res_points = np.clip(res_points,a_min=0,a_max=100.)

# print(res_points)

In [235]:
service_dict = {}

for i in range(len(df_service)):
    if (df_service.iloc[i][3] not in service_dict.keys()):
        service_dict[df_service.iloc[i][3]] = []
        
    house_no = df_service.iloc[i,0].strip().split(' - ')
    try:
        service_dict[df_service.iloc[i][3]].append(house_no[0])
        service_dict[df_service.iloc[i][3]].append(float(house_no[1]))   
    except ValueError:
        #Some random thing
        1+2
print(service_dict)

{101849: ['A1', 1001.0, 'A1', 1002.0, 'A5', 2.0, 'SE'], 100116: ['A1', 1001.0], 101813: ['A1', 1002.0, 'B1', 801.0, 'B2', 1102.0, 'B2', 401.0, 'B3', 1101.0, 'B3', 301.0, 'B3', 302.0, 'B3', 602.0, 'B5', 802.0, 'B5', 901.0, 'B6', 702.0], 100125: ['A1', 1002.0], 102993: ['A1', 1002.0], 100689: ['A1', 1002.0, 'A1', 202.0], 100756: ['A1', 1002.0, 'A1', 1502.0, 'A1', 1701.0, 'A2', 1501.0, 'A4', 201.0, 'A5', 1001.0, 'A5', 102.0, 'A5', 701.0, 'A5', 702.0, 'A5', 802.0, 'B1', 1201.0, 'B5', 1102.0], 100011: ['A1', 1002.0, 'A1', 101.0, 'A1', 1501.0, 'A1', 801.0, 'A4', 302.0], 100123: ['A1', 1002.0], 102845: ['A1', 101.0], 102898: ['A1', 101.0], 100437: ['A1', 102.0, 'A3', 501.0, 'A3', 602.0], 102508: ['A1', 102.0, 'A1', 1501.0, 'B4', 801.0], 102289: ['A1', 102.0, 'A1', 602.0, 'A2', 101.0, 'A2', 201.0, 'A3', 1101.0, 'B1', 602.0, 'B3', 1502.0, 'B4', 1101.0, 'B6', 1702.0, 'B6', 202.0, 'B6', 701.0, 'B6', 902.0], 100015: ['A1', 102.0], 101373: ['A1', 102.0], 100130: ['A1', 1101.0], 100131: ['A1', 1101.

In [236]:
#Randomise servicemen having 
service_positive = np.random.choice([0, 1], p=[0.98, 0.02], size=len(service_dict))

count = 0

for key in service_dict.keys():
    service_dict[key].append(service_positive[count])
    count += 1
    print(service_dict[key])

['A1', 1001.0, 'A1', 1002.0, 'A5', 2.0, 'SE', 0]
['A1', 1001.0, 0]
['A1', 1002.0, 'B1', 801.0, 'B2', 1102.0, 'B2', 401.0, 'B3', 1101.0, 'B3', 301.0, 'B3', 302.0, 'B3', 602.0, 'B5', 802.0, 'B5', 901.0, 'B6', 702.0, 0]
['A1', 1002.0, 0]
['A1', 1002.0, 0]
['A1', 1002.0, 'A1', 202.0, 0]
['A1', 1002.0, 'A1', 1502.0, 'A1', 1701.0, 'A2', 1501.0, 'A4', 201.0, 'A5', 1001.0, 'A5', 102.0, 'A5', 701.0, 'A5', 702.0, 'A5', 802.0, 'B1', 1201.0, 'B5', 1102.0, 0]
['A1', 1002.0, 'A1', 101.0, 'A1', 1501.0, 'A1', 801.0, 'A4', 302.0, 0]
['A1', 1002.0, 0]
['A1', 101.0, 0]
['A1', 101.0, 0]
['A1', 102.0, 'A3', 501.0, 'A3', 602.0, 0]
['A1', 102.0, 'A1', 1501.0, 'B4', 801.0, 0]
['A1', 102.0, 'A1', 602.0, 'A2', 101.0, 'A2', 201.0, 'A3', 1101.0, 'B1', 602.0, 'B3', 1502.0, 'B4', 1101.0, 'B6', 1702.0, 'B6', 202.0, 'B6', 701.0, 'B6', 902.0, 0]
['A1', 102.0, 0]
['A1', 102.0, 0]
['A1', 1101.0, 0]
['A1', 1101.0, 0]
['A1', 1101.0, 'A2', 1002.0, 'A2', 101.0, 'A3', 502.0, 'A3', 701.0, 'A4', 501.0, 'B3', 1101.0, 'B3', 602.

In [237]:
#Add 50 points to other houses if servicemen test positive
#Add 50 points to houses if cases is positive in atleast one house and servicemen are common
#Add 20 points to houses if serviceman went to house with points>50

for key in service_dict.keys():
    
    rmax = (len(service_dict[key])//2)  

    if (len(service_dict[key])%2):
        rmax-=1
    
    #If serviceman tested positive
    if(service_dict[key][-1]):
        for i in range(rmax):
            for j in range(len(res_info)):
                if(service_dict[key][2*i]==res_info[j][0] and service_dict[key][2*i+1]==res_info[j][1]):
                    res_points[j]+=50
#                     print(key)
    
    #If serviceman went to positive testing house 
    else:
        for i in range(rmax):
            change = 0
            for j in range(len(res_info)):
                if(service_dict[key][2*i]==res_info[j][0] and service_dict[key][2*i+1]==res_info[j][1]):
                    #If serviceman went to a high risk house, increase points for all houses he went to by 50
                    if(res_points[j]>=100):
                        change=50
                        break
                    #If serviceman went to a semi high risk house, increase points for all houses he went to by 20
                    elif(res_points[j]<100 and res_points[j]>=50):
                        change=20
                    
            for j in range(len(res_info)):
                if(service_dict[key][2*i]==res_info[j][0] and service_dict[key][2*i+1]==res_info[j][1]):
                    res_points[j]+=change

In [238]:
#Limit points to 100
res_points = np.clip(res_points,a_min=0,a_max=100.)

In [239]:
print(res_points)

[ 20.  20.  20.  20.  20.  20.  20.  20.  20.  20.  20.  20.  20.  80.
 100.  20.  20.  20.  20.  90.  20.  20.  20.  20.  20.  20.  20.  20.
  20.  20.  20.  20.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  60.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  20.  20.  20.  20.  20. 100.  60.  20.  20.  20.  20.  20.  20.
  20.  20.  20.  20.  20.  20.  20.  20.  20.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10. 

In [240]:
#Houses = 100 are containment zones, colour coded purple
#Houses above 50 are red
#Houses >=20 are yellow
#Rest are green

labels = []
pcount =0 
rcount =0
ycount =0
gcount =0

for i in range(len(res_points)):
    if(res_points[i]==100):
        labels.append('Purple')
        pcount+=1
    elif(res_points[i]<100 and res_points[i]>=50):
        labels.append('Red')
        rcount+=1
    elif(res_points[i]<50 and res_points[i]>=20):
        labels.append('Yellow')
        ycount+=1
    else:
        labels.append('Green')
        gcount+=1

In [241]:
print("Number of containtment zone houses are %d\n"% pcount)
print("Number of Red zone houses are %d\n"% rcount)
print("Number of Yellccow zone houses are %d\n"% ycount)
print("Number of Green zone houses are %d"% gcount)

Number of containtment zone houses are 3

Number of Red zone houses are 5

Number of Yellccow zone houses are 49

Number of Green zone houses are 270
